In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import ipywidgets as widgets
from IPython.display import display, clear_output

def load_data(filename='data.csv'):
    """
    Load data from a CSV file into a pandas DataFrame.

    Args:
    filename (str): The name of the CSV file to read from. Defaults to 'data.csv'.

    Returns:
    pandas.DataFrame: The loaded data.
    """
    df = pd.read_csv(filename)
    # Convert date and time columns to datetime
    df['datetime'] = pd.to_datetime(df['date'] + ' ' + df['time'])
    return df

def create_dynamic_visualization(filename='data.csv'):
    """
    Create a dynamic visualization with date/time range and category filters.

    Args:
    filename (str): The name of the CSV file to read from. Defaults to 'data.csv'.
    """
    # Load data
    try:
        df = load_data(filename)
    except FileNotFoundError:
        print(f"Error: File '{filename}' not found. Please check the file path.")
        return
    except Exception as e:
        print(f"Error loading data: {e}")
        return

    # Extract min and max datetime for the slider
    min_date = df['datetime'].min()
    max_date = df['datetime'].max()

    # Create widgets
    date_range_slider = widgets.SelectionRangeSlider(
        options=[(d.strftime('%Y-%m-%d %H:%M'), d) for d in pd.date_range(min_date, max_date, freq='H')],
        index=(0, len(pd.date_range(min_date, max_date, freq='H'))-1),
        description='Date Range:',
        layout=widgets.Layout(width='800px')
    )

    category_selector = widgets.SelectMultiple(
        options=['pm25', 'co', 'co2', 'tvoc'],
        value=['pm25'],
        description='Categories:',
        disabled=False
    )

    plot_button = widgets.Button(
        description='Update Plot',
        disabled=False,
        button_style='',
        tooltip='Click to update the plot',
        icon='refresh'
    )

    output = widgets.Output()

    def update_plot(b):
        with output:
            clear_output(wait=True)

            # Get selected date range
            start_date, end_date = date_range_slider.value

            # Filter data by date range
            filtered_df = df[(df['datetime'] >= start_date) & (df['datetime'] <= end_date)]

            if filtered_df.empty:
                print("No data available for the selected date range.")
                return

            # Create plot
            fig, ax = plt.subplots(figsize=(12, 6))

            for category in category_selector.value:
                ax.plot(filtered_df['datetime'], filtered_df[category], label=category)

            ax.set_xlabel('Date and Time')
            ax.set_ylabel('Value')
            ax.set_title('Air Quality Measurements')
            ax.legend()
            ax.grid(True)

            # Format x-axis to show date and time
            fig.autofmt_xdate()

            plt.show()

    # Connect the button to the update function
    plot_button.on_click(update_plot)

    # Display widgets
    display(widgets.VBox([
        widgets.HBox([date_range_slider]),
        widgets.HBox([category_selector, plot_button]),
        output
    ]))

    # Initial plot 初始
    update_plot(None)

# Example usage
# Create sample data if needed
def create_sample_data(num_days=7, filename='sample_data.csv'):
    """
    Create sample data for testing.

    Args:
    num_days (int): Number of days to generate data for. Defaults to 7.
    filename (str): The name of the CSV file to write to. Defaults to 'sample_data.csv'.

    Returns:
    None
    """
    # Create date range
    start_date = dt.datetime(2023, 1, 1)
    dates = []
    times = []
    pm25_values = []
    co_values = []
    co2_values = []
    tvoc_values = []

    for day in range(num_days):
        for hour in range(24):
            current_datetime = start_date + dt.timedelta(days=day, hours=hour)
            dates.append(current_datetime.strftime('%Y-%m-%d'))
            times.append(current_datetime.strftime('%H:%M'))

            # Generate some random values with trends
            daytime_factor = np.sin(np.pi * hour / 12) / 2 + 0.5  # Higher during day

            pm25_values.append(10 + 5 * daytime_factor + np.random.normal(0, 2))
            co_values.append(0.5 + 0.3 * daytime_factor + np.random.normal(0, 0.1))
            co2_values.append(400 + 100 * daytime_factor + np.random.normal(0, 20))
            tvoc_values.append(200 + 50 * daytime_factor + np.random.normal(0, 10))

    # Create DataFrame
    data = pd.DataFrame({
        'date': dates,
        'time': times,
        'pm25': pm25_values,
        'co': co_values,
        'co2': co2_values,
        'tvoc': tvoc_values
    })

    # Save to CSV
    data.to_csv(filename, index=False)
    print(f"Sample data created and saved to {filename}")



In [ ]:
# Uncomment below to create sample data
# create_sample_data()
# create_dynamic_visualization('sample_data.csv')

# To use with your own data
create_dynamic_visualization('data.csv')


<ipython-input-7-5feeb2e910f9>:46: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  options=[(d.strftime('%Y-%m-%d %H:%M'), d) for d in pd.date_range(min_date, max_date, freq='H')],
<ipython-input-7-5feeb2e910f9>:47: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  index=(0, len(pd.date_range(min_date, max_date, freq='H'))-1),


In [ ]:
# 若顯示 Error: File 'data.csv' not found. Please check the file path.
# 且你確定路徑正確的話運行以下腳本

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 記得更改以下路徑
create_dynamic_visualization('data.csv')